In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
from ipywidgets import Dropdown, Button, Image

## Importing the Datasets

In [2]:
roadDistance = pd.read_csv('Dataset/RoadDistance.csv')
straightDistance = pd.read_csv('Dataset/StraightDistance.csv')

### Pre-processing the datasets

In [3]:
roadDistance.index = roadDistance["Unnamed: 0"]
roadDistance.drop(["Unnamed: 0"], axis=1, inplace=True)
roadDistance.index.name = ''
##########
straightDistance.index = straightDistance["Unnamed: 0"]
straightDistance.drop(["Unnamed: 0"], axis=1, inplace=True)
straightDistance.index.name = ''

### Datasets after preprocessing

In [4]:
# straightDistance.head()
roadDistance.head()

,PashupatinathTemple,BoudhanathStupa,SwayambhunathStupa,Thamel,HanumanDhokaTemple,BuddhaNilkanth,IndraChowk,DakshinkaliTemple,NationalBotanicalGardens,SetoMachendranathTemple,...,PatanDurbarSquare,Nagarkot,Chandragiri,Bhaktapur,Dhulikhel,Kirtipur,Phulchoki,Kulekhani,NamoBuddha,Kakani
,,,,,,,,,,,,,,,,,,,,,
PashupatinathTemple,0.0,2.1,NaN,NaN,5.0,8.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,NaN,NaN
BoudhanathStupa,2.1,0.0,NaN,NaN,NaN,7.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SwayambhunathStupa,NaN,NaN,0.0,NaN,NaN,NaN,NaN,21.7,NaN,NaN,...,NaN,NaN,10.7,NaN,NaN,9.4,NaN,NaN,NaN,NaN
Thamel,NaN,NaN,NaN,0.0,1.6,NaN,1.60,NaN,NaN,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HanumanDhokaTemple,5.0,NaN,NaN,1.6,0.0,NaN,0.29,19.3,16.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def getDistance(df, from_, to_):
    dis = df.loc[from_][to_]
    return dis

In [6]:
class Greedy:
    def __init__(self, adjMat, heuristicMat, from_, to_):
        self.adjMat = adjMat
        self.from_ = from_
        self.to_ = to_
        self.heuristicMat = heuristicMat
        self.shortDistances = self.createShortestDistanceDf()
        
#         self.visited = [self.from_]
        self.costs = [0]
        self.mainAlgorithm()
    
    def mainAlgorithm(self):
        currentNode = self.from_
        self.visited = [currentNode]
        for step in range(25):
            possibleNextNodes = roadDistance[[currentNode]].query(f"{currentNode} > 0")
            for nextNode in self.shortDistances.index:
                if nextNode in possibleNextNodes.index and nextNode not in self.visited:
                    self.costs.append(getDistance(self.adjMat, currentNode, nextNode))
                    currentNode = nextNode
                    self.visited.append(currentNode)
                    break
            if currentNode == self.to_:
                break
        print(f'\nWith Greedy:\n{self.visited}\n{self.costs}\nTotal Cost = {sum(self.costs)}')
                    
    def createShortestDistanceDf(self):
        places = []
        straightDistances = []

        for node in self.heuristicMat.index:
            places.append(node)
            straightDistances.append(getDistance(straightDistance, self.to_, node))

        return pd.DataFrame({'sd': straightDistances}, index=places)['sd'].sort_values()

G = Greedy(roadDistance, straightDistance, 'Kulekhani', 'Kakani')


With Greedy:
['Kulekhani', 'DakshinkaliTemple', 'SwayambhunathStupa', 'Chandragiri']
[0, 27.5, 21.7, 10.7]
Total Cost = 59.900000000000006


In [7]:
class Astar:
    def __init__(self, adjMat, heuristicMat, from_, to_):
        self.adjMat = adjMat
        self.from_ = from_
        self.to_ = to_
        self.heuristicMat = heuristicMat
        self.shortDistances = self.createShortestDistanceDf()
        
        self.visited = [self.from_]
        self.costs = [0]
        self.mainAlgorithm()
    
    def mainAlgorithm(self):
        currentNode = self.from_
        self.visited = []
        for step in range(25):
            nextNodes = roadDistance[[currentNode]].query(f"{currentNode} > 0")
            c, p, g_h = [], [], []
            for cost, place in zip(self.shortDistances, self.shortDistances.index):
                if place in nextNodes.index and place not in self.visited:
                    c.append(cost)
                    p.append(place)
                    g = getDistance(self.heuristicMat, currentNode, place)
                    h = cost
                    g_h.append(g+h)
            self.g_hCostDf = pd.DataFrame({'g_h':g_h}, index=p).sort_values('g_h')
            nextNode = self.g_hCostDf.index[0]
            self.visited.append(currentNode)
            self.costs.append(getDistance(self.heuristicMat, currentNode, nextNode))
            currentNode = nextNode

            
            if currentNode == self.to_:
                self.visited.append(currentNode)
                break
        print(f'\nWith A*:\nVisited: {self.visited}\nCorresponding Cost: {self.costs}\nTotal Cost = {sum(self.costs)}')
                    
    def createShortestDistanceDf(self):
        places = []
        straightDistances = []

        for node in self.heuristicMat.index:
            places.append(node)
            straightDistances.append(getDistance(straightDistance, self.to_, node))

        return pd.DataFrame({'sd': straightDistances}, index=places)['sd'].sort_values()

A = Astar(roadDistance, straightDistance, 'Kulekhani', 'Kakani')


With A*:
Visited: ['Kulekhani', 'DakshinkaliTemple', 'HanumanDhokaTemple', 'PashupatinathTemple', 'BoudhanathStupa', 'BuddhaNilkanth', 'Kakani']
Corresponding Cost: [0, 9.96, 0.257, 4.12, 1.68, 4.91, 13.59]
Total Cost = 34.516999999999996


In [24]:
destination = Dropdown(description = "Destination", options=straightDistance.index)
source = Dropdown(description = "Source", options=straightDistance.index.drop(destination.value))
AS = Button(description="Search With A*")
GA = Button(description="Search With Greedy")
display(source)
display(destination)
display(AS)
display(GA)

def searchWithAstar(arg):
    Astar(roadDistance, straightDistance, source.value, destination.value)
def searchWithGreedy(arg):
    Greedy(roadDistance, straightDistance, source.value, destination.value)

AS.on_click(searchWithAstar)
GA.on_click(searchWithGreedy)

#Kulekhani to Kakani Greedy's Algo defect

Dropdown(description='Source', options=('BoudhanathStupa', 'SwayambhunathStupa', 'Thamel', 'HanumanDhokaTemple…

Dropdown(description='Destination', options=('PashupatinathTemple', 'BoudhanathStupa', 'SwayambhunathStupa', '…

Button(description='Search With A*', style=ButtonStyle())

Button(description='Search With Greedy', style=ButtonStyle())

<img src="KathmanduMap.png" alt="drawing" width="70%"/>